In [1]:
# Get news title and paragraph text

In [2]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [3]:
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\Corey Anderson\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


<h3> Q1: get latest title and paragraph body from redplanet.science.com </h3>

In [5]:
mars_url = "https://redplanetscience.com/"

In [6]:
browser.visit(mars_url)

In [7]:
html = browser.html
soup = bs(html, 'html.parser')

In [8]:
latest_titles = soup.find_all("div", class_ = "content_title")

In [9]:
print(latest_titles)

[<div class="content_title">NASA's Mars Reconnaissance Orbiter Undergoes Memory Update</div>, <div class="content_title">Common Questions about InSight's 'Mole'</div>, <div class="content_title">The Detective Aboard NASA's Perseverance Rover</div>, <div class="content_title">NASA's Mars 2020 Will Hunt for Microscopic Fossils</div>, <div class="content_title">NASA's New Mars Rover Will Use X-Rays to Hunt Fossils</div>, <div class="content_title">10.9 Million Names Now Aboard NASA's Perseverance Mars Rover</div>, <div class="content_title">Join NASA for the Launch of the Mars 2020 Perseverance Rover</div>, <div class="content_title">NASA, ULA Launch Mars 2020 Perseverance Rover Mission to Red Planet</div>, <div class="content_title">NASA Updates Mars 2020 Mission Environmental Review</div>, <div class="content_title">Independent Review Indicates NASA Prepared for Mars Sample Return Campaign</div>, <div class="content_title">NASA's InSight Flexes Its Arm While Its 'Mole' Hits Pause</div>,

In [10]:
news_title = latest_titles[0].text
print(news_title)

NASA's Mars Reconnaissance Orbiter Undergoes Memory Update


In [11]:
latest_p = soup.find_all("div", class_ = "article_teaser_body")
news_p  = latest_p[0].text
print(news_p)

Other orbiters will continue relaying data from Mars surface missions for a two-week period.


<h3> Q2: get url for featured picture image on spaceimages-mars.com </h3>

In [12]:
pic_url = "https://spaceimages-mars.com/#"
browser.visit(pic_url)
html = browser.html
soup = bs(html, 'html.parser')

In [13]:
# feature_image_url

test = soup.find_all("img", class_ ="headerimage fade-in")
print(test)

url_end = test[0]["src"]
feature_image_url = f"https://spaceimages-mars.com/{url_end}"

print(feature_image_url)

[<img class="headerimage fade-in" src="image/featured/mars2.jpg"/>]
https://spaceimages-mars.com/image/featured/mars2.jpg


<h3> Q3: scrape table on Mars facts from galaxyfacts-mars.com </h3>.
<p> Note that the table in transposed (i.e. column headers are row headers) </p> 

In [14]:
space_url = 'https://galaxyfacts-mars.com/'

In [15]:
browser.visit(space_url)
html = browser.html
soup = bs(html, 'html.parser')

In [16]:
soup.find("table", class_="table table-striped")

<table class="table table-striped">
<tbody>
<tr>
<th scope="row">Equatorial Diameter:</th>
<td>6,792 km</td>
</tr>
<tr>
<th scope="row">Polar Diameter:</th>
<td>6,752 km</td>
</tr>
<tr>
<th scope="row">Mass:</th>
<td>	6.39 × 10^23 kg (0.11 Earths)</td>
</tr>
<tr>
<th scope="row">Moons:</th>
<td>	2 ( <span class="red">Phobos </span> &amp; <span class="red"> Deimos </span>)</td>
</tr>
<tr>
<th scope="row">Orbit Distance:</th>
<td>	227,943,824 km (1.38 AU)</td>
</tr>
<tr>
<th scope="row">Orbit Period:</th>
<td>	687 days (1.9 years)</td>
</tr>
<tr>
<th scope="row">Surface Temperature:</th>
<td>	-87 to -5 °C</td>
</tr>
<tr>
<th scope="row">First Record:</th>
<td>	2nd millennium BC</td>
</tr>
<tr>
<th scope="row">Recorded By:</th>
<td>	Egyptian astronomers</td>
</tr>
</tbody>
</table>

In [17]:
mars_table = soup.find("table", class_="table table-striped")

In [18]:
mars_tr = mars_table.find_all("tr")

In [19]:
col_names = [None] * 9
mars_data = [None] * 9

for i in range(0, 9):
    col_names[i] = mars_tr[i].th.text.strip()
    mars_data[i] = mars_tr[i].td.text.strip()

print(col_names)
print(mars_data)

['Equatorial Diameter:', 'Polar Diameter:', 'Mass:', 'Moons:', 'Orbit Distance:', 'Orbit Period:', 'Surface Temperature:', 'First Record:', 'Recorded By:']
['6,792 km', '6,752 km', '6.39 × 10^23 kg (0.11 Earths)', '2 ( Phobos  &  Deimos )', '227,943,824 km (1.38 AU)', '687 days (1.9 years)', '-87 to -5 °C', '2nd millennium BC', 'Egyptian astronomers']


In [20]:
col_names_cleaned = [th.replace(":", "") for th in col_names]
print(col_names_cleaned)

['Equatorial Diameter', 'Polar Diameter', 'Mass', 'Moons', 'Orbit Distance', 'Orbit Period', 'Surface Temperature', 'First Record', 'Recorded By']


In [21]:
mars_df = pd.DataFrame(mars_data)

In [22]:
mars_df_transposed = mars_df.transpose()

In [23]:
display(mars_df_transposed)

,0,1,2,3,4,5,6,7,8
0,"6,792 km","6,752 km",6.39 × 10^23 kg (0.11 Earths),2 ( Phobos & Deimos ),"227,943,824 km (1.38 AU)",687 days (1.9 years),-87 to -5 °C,2nd millennium BC,Egyptian astronomers


In [24]:
mars_df_transposed.columns = col_names_cleaned

In [25]:
mars_df_transposed["Planet"] = "Mars"

In [26]:
mars_df_transposed.set_index("Planet")

,Equatorial Diameter,Polar Diameter,Mass,Moons,Orbit Distance,Orbit Period,Surface Temperature,First Record,Recorded By
Planet,,,,,,,,,
Mars,"6,792 km","6,752 km",6.39 × 10^23 kg (0.11 Earths),2 ( Phobos & Deimos ),"227,943,824 km (1.38 AU)",687 days (1.9 years),-87 to -5 °C,2nd millennium BC,Egyptian astronomers


In [27]:
table_html = mars_df_transposed.to_html()

In [28]:
table_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Equatorial Diameter</th>\n      <th>Polar Diameter</th>\n      <th>Mass</th>\n      <th>Moons</th>\n      <th>Orbit Distance</th>\n      <th>Orbit Period</th>\n      <th>Surface Temperature</th>\n      <th>First Record</th>\n      <th>Recorded By</th>\n      <th>Planet</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>6,792 km</td>\n      <td>6,752 km</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n      <td>2 ( Phobos  &amp;  Deimos )</td>\n      <td>227,943,824 km (1.38 AU)</td>\n      <td>687 days (1.9 years)</td>\n      <td>-87 to -5 °C</td>\n      <td>2nd millennium BC</td>\n      <td>Egyptian astronomers</td>\n      <td>Mars</td>\n    </tr>\n  </tbody>\n</table>'

<h3> Q4: marshemisphers.com </h3>

<p> Gather image urls.</p>

In [29]:
hemi_url = "https://marshemispheres.com/"

In [30]:
browser.visit(hemi_url)
html = browser.html
soup = bs(html, 'html.parser')

In [31]:
soup

<html lang="en"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="css/jquery-ui.css" rel="stylesheet" type="text/css"/>
<title>Astropedia Search Results | GUSS Astrogeology Science Center</title>
<meta content="GUSS Astrogeology Science Center Astropedia search results." name="description"/>
<meta content="GUSS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping" name="keywords"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<link href="css/main.css" media="screen" rel="stylesheet"/>
<link href="css/print.css" media="print" rel="stylesheet"/>
<link href="#" rel="icon" type="image/x-ico"/>
</head>
<body id="results">
<header>
<a href="#" style="float:right;margin-top:10px;" target="_blank">
<img alt="USGS: Science for a Changing World" class="logo" height="60" src="images/usgs_logo_main_2x.png"/>
</a>
<a href="#" style="float:

In [32]:
descriptions = soup.find_all("div", class_ = "description")

In [33]:
urls = []

for i in range(len(descriptions)):
    piece = descriptions[i].a["href"]
    urls.append(f"https://marshemishperes.com/{piece}")
    
print(urls)

['https://marshemishperes.com/cerberus.html', 'https://marshemishperes.com/schiaparelli.html', 'https://marshemishperes.com/syrtis.html', 'https://marshemishperes.com/valles.html']


In [34]:
pic_names = []

for j in range(len(descriptions)):
    pic_names.append(descriptions[j].h3.text.strip())

print(pic_names)

['Cerberus Hemisphere Enhanced', 'Schiaparelli Hemisphere Enhanced', 'Syrtis Major Hemisphere Enhanced', 'Valles Marineris Hemisphere Enhanced']


In [35]:
hemisphere_image_urls = {}

In [36]:
for k in range(len(descriptions)):
    hemisphere_image_urls[pic_names[k]] = urls[k]

print(hemisphere_image_urls)

{'Cerberus Hemisphere Enhanced': 'https://marshemishperes.com/cerberus.html', 'Schiaparelli Hemisphere Enhanced': 'https://marshemishperes.com/schiaparelli.html', 'Syrtis Major Hemisphere Enhanced': 'https://marshemishperes.com/syrtis.html', 'Valles Marineris Hemisphere Enhanced': 'https://marshemishperes.com/valles.html'}
